In [4]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime, timedelta


from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import tqdm
from tqdm.notebook import tqdm

In [6]:
query_txt='디즈니플러스 '

## 대표프로그램 에 대한 사용자 컨텐츠 가져오기

In [3]:
disneey_df=pd.read_excel(r"D:\Python\TM\TeamProject/디즈니플러스.xlsx")
disneey_df

,제목,장르
0,만달로리안,스페이스 오페라
1,필사의 도전,역사 드라마
2,완다비전,"슈퍼히어로, 시트콤"
3,팔콘과 윈터 솔져,"슈퍼히어로, 액션"
4,로키,"슈퍼히어로, 판타지"
...,...,...
59,비하인드 어트랙션,다큐멘터리
60,자라나는 동물들,자연
61,스파크 스토리,다큐멘터리
62,우주 속으로,다큐멘터리


In [7]:
query_list= disneey_df["제목"].tolist()
query_list = [query_txt + " " + i for i in query_list]
print(query_list)

['디즈니플러스  만달로리안', '디즈니플러스  \xa0필사의 도전', '디즈니플러스  완다비전', '디즈니플러스  팔콘과 윈터 솔져', '디즈니플러스  로키', '디즈니플러스  베네딕트 비밀클럽', '디즈니플러스  천재소녀 두기', '디즈니플러스  호크아이', '디즈니플러스  하이 스쿨 뮤지컬: 뮤지컬: 시리즈', '디즈니플러스  나의 대통령 일기', '디즈니플러스  마이티 덕: 게임 체인저', '디즈니플러스  빅샷', '디즈니플러스  터너와 후치', '디즈니플러스  저스트 비욘드', '디즈니플러스  디즈니 얽히고 엮인 그녀들', '디즈니플러스  재미있는 디즈니 공작교실', '디즈니플러스  앙코르! 다시 무대로', '디즈니플러스  픽사', '디즈니플러스  리얼 라이프, 픽사', '디즈니플러스  샵 클래스: 어린 공학자', '디즈니플러스  도전! 디즈니 셰프', '디즈니플러스  빅 핍: 누구의 거짓말?', '디즈니플러스  머펫 나우', '디즈니플러스  지구에서 네드에게', '디즈니플러스  로빈 로버츠: 세상을 토론하다', '디즈니플러스  런닝맨: 뛰는 놈 위에 노는 놈', '디즈니플러스  포키는 궁금한 게 많아요', '디즈니플러스  스파크쇼츠', '디즈니플러스  디즈니 애니메이션 스튜디오 단편 모음', '디즈니플러스  제니메이션', '디즈니플러스  미키마우스의 즐거운 세상', '디즈니플러스  픽사 팝콘', '디즈니플러스  스타워즈: 배드 배치', '디즈니플러스  몬스터 근무일지', '디즈니플러스  칩 앤 데일: 파크 라이프', '디즈니플러스  What If...?', '디즈니플러스  구피가 집에서 잘 지내는 방법', '디즈니플러스  더그의 일상', '디즈니플러스  스타워즈: 비전스', '디즈니플러스  올라프가 전해요', '디즈니플러스  제프 골드블럼의 호기심 세계', '디즈니플러스  이매지니어링 스토리', '디즈니플러스  마블의 영웅 프로젝트', '디즈니플러스  디즈니의 평범한 하루 (단편)', '디즈니플러스  도전! 안내견', '디즈니플러스  디즈니 인사이더

### step 1. 대표프로그램 중 50개의 프로그램에 대해서 블로그 검색 URL  가져오기

In [8]:
data_param={}

for title in query_list[:50] :
    #Step 1. 크롬 웹브라우저 실행
    path = "D:\Python\TM\drivers\chromedriver.exe"

    driver = webdriver.Chrome(r"D:\Python\TM\drivers\chromedriver.exe")
    # 사이트 주소는 네이버
    driver.get('http://www.naver.com')
    time.sleep(2)
    
    try : 
    
        #Step 2. 네이버 검색창에 "검색어" 검색
        element = driver.find_element_by_id("query")
        element.send_keys(title)  
        element.submit()

        time.sleep(2)
        #Step 3. "VIEW" 카테고리 선택
        driver.find_element_by_link_text("VIEW").click( )
        time.sleep(2)
        #Step 3-1. "블로그" 카테고리 선택
        driver.find_element_by_link_text("블로그").click( )    # .click() 괄호 안을 눌러라는 뜻

        time.sleep(2)

        #Step 4. 오른쪽의 검색 옵션 버튼 클릭
        driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[2]/a').click( )

        #Step 5. 정렬 : "관련도순" 
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[2]/div/div/a[1]').click( )  # 정렬 버튼의 xpath 클릭

        time.sleep(2)

        #Step 6. 날짜 입력을 위해서 옵션 재클릭 후 " 기간 6개월 선택"
        driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[2]/a').click( )
        time.sleep(2)
        #driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[7]').click()
        driver.find_element(By.XPATH,'//*[contains(text(),"6개월") and @class="txt"]').click()
        data_param[title]=driver.find_element(By.XPATH,'//*[contains(text(),"6개월") and @class="txt"]').get_attribute('data-param')
        
    except :
        driver.close()
        time.sleep(1)
        continue
        
    driver.close()
    

<ipython-input-8-1f17cd855318>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"D:\Python\TM\drivers\chromedriver.exe")
<ipython-input-8-1f17cd855318>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
<ipython-input-8-1f17cd855318>:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text("VIEW").click( )
<ipython-input-8-1f17cd855318>:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text("블로그").click( )    # .click() 괄호 안을 눌러라는 뜻
<ipython-input-8-1f17cd855318>:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[2]/a').click( )
<ipython-input-8-1f17cd8

### step 2 대표프로그램 중 50개의 프로그램에 대해서 블로그 검색 및 URL 정보에 들어가서 각 블로그 정보 크롤링

In [9]:
def find_blogPage(movies_titles) :
    url_list = []
    title_list = []
    date_list = []
    driver = webdriver.Chrome(r"D:\Python\TM\drivers\chromedriver.exe")
    
    for k, v in movies_titles.items():
        # ★몇개의 페이지를 크롤링할지 선택
        total_page = 2 
        for i in tqdm(range(1, total_page)):  # 페이지 번호
            #i = i*10 + 1
            print(i)
            url = "https://search.naver.com/search.naver?&query={0}&sm=tab_opt&{1}&where=post&start={2}".format("+".join(k.split()),v,i)
            print(url)
            driver.get(url)
            time.sleep(2)

            # URL 크롤링 시작
            article_raw = driver.find_elements(By.XPATH, "//a[@class= 'api_txt_lines total_tit']")

            # url 크롤링 시작    
            for article in article_raw:
                url = article.get_attribute('href')   
                url_list.append(url)

            # 제목 크롤링 시작    
            for article in article_raw:
                title = article.text
                title_list.append(title)


            # 업로드 날짜 크롤링 시작
            article_date = driver.find_elements(By.XPATH,'//span[@class= "sub_time sub_txt"]')

            for artice in article_date :
                if "전" in artice.text  or artice.text in "어제" :
                    if "일" in artice.text   :
                        date = (datetime.today() - timedelta(int(artice.text[0]))).strftime("%Y.%m.%d.")
                    elif "어제" in artice.text :
                        date=(( datetime.today() - timedelta(1)).strftime("%Y.%m.%d.")) 
                    else :
                        date=datetime.now().strftime("%Y.%m.%d")

                else :
                    date  = artice.text
                date_list.append(date)
                
                
    print('url갯수: ', len(url_list))
    print('url갯수: ', len(title_list))
    print('url갯수: ', len(date_list))

    df = pd.DataFrame({'url':url_list, 'title':title_list, 'date' : date_list })
    
    #driver.close()
    return df

In [10]:
    # 저장하기
df = find_blogPage(data_param)
df.to_excel(r"D:\Python\TM\TeamProject/disney_6month.xlsx")

<ipython-input-9-443d4b312203>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"D:\Python\TM\drivers\chromedriver.exe")


  0%|          | 0/1 [00:00<?, ?it/s]

1
https://search.naver.com/search.naver?&query=디즈니플러스+만달로리안&sm=tab_opt&nso=so%3Ar%2Cp%3A6m&where=post&start=1


  0%|          | 0/1 [00:00<?, ?it/s]

1
https://search.naver.com/search.naver?&query=디즈니플러스+필사의+도전&sm=tab_opt&nso=so%3Ar%2Cp%3A6m&where=post&start=1


...

1
https://search.naver.com/search.naver?&query=디즈니플러스+디즈니+갤러리+/+스타워즈:+만달로리안&sm=tab_opt&nso=so%3Ar%2Cp%3A6m&where=post&start=1


  0%|          | 0/1 [00:00<?, ?it/s]

1
https://search.naver.com/search.naver?&query=디즈니플러스+숨겨진+세상:+겨울왕국2+메이킹&sm=tab_opt&nso=so%3Ar%2Cp%3A6m&where=post&start=1


  0%|          | 0/1 [00:00<?, ?it/s]

1
https://search.naver.com/search.naver?&query=디즈니플러스+별난+여행&sm=tab_opt&nso=so%3Ar%2Cp%3A6m&where=post&start=1
url갯수:  1129
url갯수:  1129
url갯수:  1129


## 본문 크롤링 하기

### step 3 대표프로그램 중 50개의 프로그램에 대해서 건별 블로그에 내용 GET(본문크롤링 => 자세한 분석은 code3)

In [11]:
# "url_list.csv" 불러오기
url_load = pd.read_excel(r"D:\Python\TM\TeamProject/disney_6month.xlsx")        # 기본 모델
#url_load =df
print(len(url_load))


1129


In [123]:
#340+48
chk_url = url_load['url'][388]
print(chk_url.split("logNo="))

['https://anilover.tistory.com/5835']


In [13]:
dict={}
number = 1100  #크롤링하고자하는 블로그 글의 갯수
for i in tqdm(range(700, number)): 
    # 글 띄우기
    url = url_load['url'][i]
    #print(url.split("logNo=")[1])
    driver = webdriver.Chrome(r"D:\Python\TM\drivers\chromedriver.exe")
    driver.get(url)   # 글 띄우기
    print(driver.get)
     # 크롤링
    time.sleep(1)
    
    try : 
        driver. switch_to.frame('mainFrame')

        target_info = {}
        # 제목 크롤링 시작
        #tit = driver.find_element_by_css_selector(overlays)         # title
        tit=driver.find_element(By.XPATH,'//meta[@property="og:title"]')
        #title = tit.text .get_attribute("")
        title = driver.find_element(By.XPATH,'//meta[@property="og:title"]').get_attribute("content")
        # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동

        # 글쓴이 크롤링 시작                                 
        #nick = driver.find_element_by_css_selector(overlays)         # nick
        nick=driver.find_element(By.XPATH,'//meta[@property="naverblog:nickname"]')
        #nickname = nick.text
        nickname = driver.find_element(By.XPATH,'//meta[@property="naverblog:nickname"]').get_attribute("content")

        # 날짜 크롤링
        datetime =url_load['date'][i]                              

        # 내용 크롤링
        # 네이버 블로그 포스팅 버전 1. #postViewArea
        # 네이버 블로그 포스팅 버전 2. #post-view222566374177 > div > div > div.se-main-container 
        soups = BeautifulSoup(driver.page_source, "lxml") 

        if soups.find("div", attrs={"class":"se-main-container"}):
            contents = soups.find("div", attrs={"class":"se-main-container"}).get_text()
            contents = contents.replace("\n","") #공백 제거

        elif soups.find("div", attrs={"id":"postViewArea"}):
            contents = soups.find("div", attrs={"id":"postViewArea"}).get_text()
            contents = contents.replace("\n","") 
        else:
            contents =  "네이버 블로그는 맞지만, 확인불가"
            
    # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i% 30 == 0  or i% 50 == 0 or i%80 == 0:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel(r"D:\Python\TM\TeamProject/disney_6month_1.xlsx")
        time.sleep(3)
    
    
    # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
    target_info['title'] = title
    target_info['nickname'] = nickname
    target_info['datetime'] = datetime
    target_info['content'] = contents    

    # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
    dict[i] = target_info
    time.sleep(1)
        
driver.close()        
result_df = pd.DataFrame.from_dict(dict, 'index')



# 저장하기
result_df.to_excel(r"D:\Python\TM\TeamProject/disney_6month_1.xlsx")


  0%|          | 0/400 [00:00<?, ?it/s]

<ipython-input-13-69169609b946>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"D:\Python\TM\drivers\chromedriver.exe")


<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="31d73875e08b2c05dd6fde2516f36f8f")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="e2c7b7d9fb5957428350f500a1338dad")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="76a8c3954c0c9a41011f517fa03ffc8d")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="124b19424449a836efb21e91d0c5b391")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="636488a375883b13185d2f566528fc82")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="af3180337cc0fb759a86e5b45a75dc20")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="64127134986dfc640f3c86009bb1bf98")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="04d95adf42d126b523a0ad939058aa59")>>


<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="4be19e30efbcfc6d8febe6a2ad06e832")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="c7121d2bafa7cd8229a7a42edc056db4")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="6167525791f7d39af886a95fa68f8b99")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="2b0d1dd28b73ead36141e5c511d95676")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="2c3f7d519963e92eea56f58457219370")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="8813369ce8561d11fd3146ac5b703fc2")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="8d9845c55522ee017396457391a30d31")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="2d310827d62f8ff147d49e4afb528ed7")>>


<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="16c1d0bb01c7d0793b5d0ff88f404fc5")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="8cf483fb11e68a64f91ddaef9984d451")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="2049188f5b5be5095b07b4ea48990686")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="5fef23fa0810f1906bebc4c3dd719c01")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="2828aec4cfda53b809fe6a2797dad348")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="c421d603fbd8f30ed3bbbffa1c8f63f4")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="bbd98f4e3b9e66248367a02fb354cc15")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="a3e6eecc2c45070547a9d01508e87cf7")>>


<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="73cf6b39895014bd9ce3e6fd75e91aa4")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="8e786e95a8a2da36676423408d9bffcb")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="dcf09139d1ff69f7eb355bc5d9423047")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="1c80deaa228e89c3a72d0aef7076cbb7")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="5102608c98021ccf9259b70352ad682c")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="39a11c0f885b720fc13f98f461561a8d")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="c72316fe9d0765affb400e48b527e810")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="85c2dbba11a3d686069f8919d2d52d65")>>


<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="afc72232b8f6c99c51b19670c510451f")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="7aa5c46b81deba35f0a6ffb17d9a3fb4")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="f21fb9ef3c2bfa60f8420a30c1f5e558")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="b810dcd696c953ddbffe5ea7c3585454")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="3c47ed18cd3593c5a1b2138d4bef5f9a")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="4120dd713973ebf1f3e549f74cea9f91")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="7dfb669cbc9d917ffb5b379061a81482")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="9843eedf468f22bdfa68fda33271f8c1")>>


<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="d3a3f416d531154f4a673e7acdd5bcb7")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="87e2c82f7d9c6fdf8c5b0f8144b20477")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="16745537bf936a44760a6ffb08e1130d")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="e1f6ae134b13df613c838dd627768a84")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="11e4b80d8dff82ee8b2da2adb3815a27")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="d8851e950c8b9d561926d723315030a7")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="5db12da4e433872b768315454b9a4e32")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="50ac3ca3dc0d6fec109b3ed4a1b1620a")>>


<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="38d9403037da69ac5c067581ea6e8942")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="b6d59d44da3a85570436cdbdc3366c60")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="f1d475f928adf3908b6e4d4b72fee2c8")>>
<bound method WebDriver.get of <selenium.webdriver.chrome.webdriver.WebDriver (session="94dfdb35ee0c37de98a228d78026380c")>>


In [2]:
from glob import glob
def grabFileName(filename) :
    
    filetext = r"D:\Python\TM\TeamProject/" + filename +"*.xlsx"
    file_names = glob(filetext) #폴더 내의 모든 csv파일 목록을 불러온다
    total = pd.DataFrame() #빈 데이터프레임 하나를 생성한다

    for file_name in file_names:
        temp = pd.read_excel(file_name) 
        total = pd.concat([total, temp]) #전체 데이터프레임에 추가하여 넣는다

    total.to_excel(r"D:\Python\TM\TeamProject/" + filename+ "all.xlsx")

In [5]:
grabFileName("disney_6month")